In [ ]:
!pip install jsonlines

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os
import json
import jsonlines
import pandas as pd

from collections import defaultdict

In [ ]:
def data_add_test(path_source, path_target, path_alignment, path_mapping):
    
    unique_doc_ids_source = lang_data_parse(path_source)
    unique_doc_ids_target = lang_data_parse(path_target)
    
    with open(path_mapping, 'r') as f:
        mapping = f.readlines()
        
    mapping = [x.strip('\n').split('\t') for x in mapping]
    mapping_dict = {}
    
    for el in mapping:
        mapping_dict[el[0]] = el[1]
        
    with open(path_alignment, 'r') as f:
        align = f.readlines()
        
    align = [x.strip('\n').split('\t') for x in align]
    align = [[mapping_dict[x[1]], x[2], x[3]] for x in align]

    return align, unique_doc_ids_source, unique_doc_ids_target

In [ ]:
def lang_data_parse(path):
    
    with open(path, 'r') as f:
        lang = f.readlines()
        
    lang = [x.split('\t') for x in lang]
    
    unique_doc_ids = defaultdict(dict)
    
    for el in lang:
        doc_id = el[0]
        unique_doc_ids[doc_id][el[1]] = el[2].strip('\n')
        
    return unique_doc_ids

In [ ]:
def generate_translation_json(path_source, path_target, path_alignment, path_mapping):
  align_20, unique_doc_ids_20, unique_doc_ids_fr_20 = data_add_test(
                            path_source,
                            path_target,
                            path_alignment,
                            path_mapping
                           )
  res_20 = []

  for el in align_20:
      if 'omitted' not in el:
          try:
              doc_id = el[0]
              sent_en = el[2]
              sent_fr = el[1]
              en = unique_doc_ids_20[doc_id][sent_en]
              fr = unique_doc_ids_fr_20[doc_id][sent_fr]
              res_20.append({"translation": {"fr": fr, "en": en}})
          except KeyError:
              pass

  return res_20



In [ ]:
mapping_original = generate_translation_json(
                            '/content/data/medline_fr2en_en.txt',
                            '/content/data/medline_fr2en_fr.txt',
                            '/content/data/fr-en_align_validation.tsv',
                            '/content/data/fren_mapping.txt'
                           )

In [ ]:
with jsonlines.open('mapping_original.jsonl', mode='w') as writer:
        writer.write(mapping_original)

In [ ]:
mapping_predictions = generate_translation_json(
                            '/content/data/medline_fr2en_fr.txt',
                            '/content/data/medline_fr2en_en_predictions.txt',
                            '/content/data/fr-en_align_validation.tsv',
                            '/content/data/fren_mapping.txt'
                           )

In [ ]:
with jsonlines.open('mapping_predictions.jsonl', mode='w') as writer:
        writer.write(mapping_predictions)